In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle 


In [2]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
#Preprocess data
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [4]:
#Encode categorical variable
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

In [6]:
#One-hot encoder 'Geography'
onehot_encoder_geo=OneHotEncoder(handle_unknown='ignore')
geo_encoded=onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [7]:
#Combine one-hot encoded columns with original data
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [8]:
#Split the data into features and target 
X=data.drop('EstimatedSalary',axis=1) 
y=data['EstimatedSalary']

In [9]:
#Split the data in training and testing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [10]:
#scale these feature
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [11]:
#Save the encoders and scaler for later use
with open ('label_encoder_gender.pkl','wb')as file:
    pickle.dump(label_encoder_gender,file)
with open ('onehot_encoder_geo.pkl','wb')as file:
    pickle.dump(onehot_encoder_geo,file)
with open ('scaler.pkl','wb')as file:
    pickle.dump(scaler,file)

ANN regression problem statement

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [15]:
#Build Our ann model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), #HL1 Connection with input layers
    Dense(32,activation='relu'), #HL2
    Dense(1)#output layers for regression 
    
]
)
#Compile the model 
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
#Set up the TensorBoard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime
log_dir="regressionlogs/fit"+datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)


In [17]:
#Set up early stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)



In [18]:
#Training the model 
history=model.fit(
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
    
)

Epoch 1/100


250/250 [==============================] - 4s 6ms/step - loss: 100369.4297 - mae: 100369.4297 - val_loss: 98478.3672 - val_mae: 98478.3672
Epoch 2/100
250/250 [==============================] - 1s 5ms/step - loss: 99454.8984 - mae: 99454.8984 - val_loss: 96611.6719 - val_mae: 96611.6719
Epoch 3/100
250/250 [==============================] - 1s 6ms/step - loss: 96203.2266 - mae: 96203.2266 - val_loss: 91848.2109 - val_mae: 91848.2109
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 89882.5469 - mae: 89882.5469 - val_loss: 84118.3594 - val_mae: 84118.3594
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 80915.6016 - mae: 80915.6016 - val_loss: 74526.4062 - val_mae: 74526.4062
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 70982.6016 - mae: 70982.6016 - val_loss: 65240.4219 - val_mae: 65240.4219
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 62161.5391 - mae: 62161.539

In [19]:
%load_ext tensorboard

In [22]:
%tensorboard --logdir regressionlogs/fit20250702-203801

Reusing TensorBoard on port 6007 (pid 10072), started 0:00:03 ago. (Use '!kill 10072' to kill it.)

In [24]:
#Evaluate model on the test data
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test MAE:{test_mae}')

63/63 [==============================] - 0s 3ms/step - loss: 50289.1680 - mae: 50289.1680
Test MAE:50289.16796875


In [25]:
model.save('regression_model.h5')

c:\Users\Kush\Desktop\Project\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
